In [5]:
import numpy as np
from autogluon.tabular import TabularPredictor
import pandas as pd
from tqdm.notebook import tqdm
import os
import math
import multiprocessing
import ctypes
from getS50 import getS50
from astropy import units as u
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


In [1]:
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def getsavefilename(SavePath, path):
    return os.path.join(SavePath, os.path.basename(path))


def corrdinateYBJ(theta, phi, mjd):
    """Transform YBJ ALTAZ to J2000

    Args:
        theta (np.array): zenith angle in location coordinates
        phi (np.array): Azimuth angle in location corrdinates
        mjd (np.array): mjd
    Returns:
        np.array: List of fo8ur np.array, first is ra with degree, second is dec with degree,third is hour angle,forth is LST.
    """
    YBJ_Location = EarthLocation(
        lat=30.102 * u.deg, lon=90.522 * u.deg, height=4300 * u.m
    )
    mjdtime = Time(mjd, format="mjd", location=YBJ_Location)
    newAltAzcoordiantes = SkyCoord(
        alt=(90 - theta) * u.deg,
        az=phi * u.deg,
        obstime=mjdtime,
        frame="altaz",
        location=YBJ_Location,
    )
    LST = mjdtime.sidereal_time("apparent").degree
    RaDec =newAltAzcoordiantes.icrs 
    Ra = RaDec.ra.degree
    Dec = RaDec.dec.degree
    ha = LST-Ra
    return [
        Ra,
        Dec,
        LST,
        ha
    ]

In [2]:
DataPath = "/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_ag_pgamma"
SavePath_merged = "/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged"
SavePath_afterag = "/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_afterag"
SavePath_RaDec = "/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec"

In [4]:
mkdir(SavePath_merged)

for root, dirs, _ in os.walk(DataPath):
    for dirname in dirs:
        print(dirname)
        Exptdata = dict()
        for root2, _, files in os.walk(os.path.join(root, dirname)):
            for filename in files:
                Exptdatatmp = np.load(os.path.join(root2, filename))
                Exptdatatmp = {key: Exptdatatmp[key] for key in Exptdatatmp}
                Exptdatatmp["S50"] = getS50(Exptdatatmp["ne"], Exptdatatmp["age"])
                for key in Exptdatatmp.keys():
                    if key not in Exptdata.keys():
                        Exptdata[key] = list()
                    Exptdata[key].append(Exptdatatmp[key])
        for key in Exptdata.keys():
            Exptdata[key] = np.concatenate(Exptdata[key])
        np.savez(os.path.join(SavePath_merged, dirname) + ".npz", **Exptdata)

003991
004103
004091
004108
004023
004058
004005
004029
004048
003972
004060
004094
004047
004021
003964
004022
004045
004102
004117
004084
004089
004085
004095
004098
003960
004027
004079
004104
004006
004040
004118
004059
004038
004101
004122
003966
004119
003971
003969
004075
003997
004003
004125
004036
004009
003985
004114
004017
004124
004002
004035
004013
004016
004115
004092
004050
004082
004054
003973
003962
004020
004019
003986
004026
003995
004024
004051
004076
003967
004081
004046
004031
004001
004107
004052
004127
004000
003984
004128
003988
003983
003963
004063
003976
004116
004015
004018
004130
004014
004121
004112
004120
004096
004067
004039
004042
004100
003980
004090
004132
004105
003977
003990
004078
003993
004004
004011
004028
003994
004099
004053
004008
004088
004065
003987
004087
003998
004066
004057
004056
004083
003970
004041
003979
003975
004106
004049
004032
004113
004093
004034
004077
003996
004111
004126
004123
004025
004086
004080
003981
004109
004033
004097

In [4]:
Pgamma_high_predictor = TabularPredictor.load("./autogloun_model/highEnergy_fewcolumn/")
Energy_high_predictor = TabularPredictor.load("./autogloun_model/highEnergy_Energy")
deltatheta_high_predictor = TabularPredictor.load(
    "./autogloun_model/highEnergy_deltatheta/"
)
deltaphi_high_predictor = TabularPredictor.load(
    "./autogloun_model/highEnergy_deltaphi/"
)

In [5]:
datalist = list()
for root, dirs, files in os.walk(SavePath_merged):
    for name in files:
        filename = os.path.join(root, name)
        datalist.append(filename)

In [6]:
for filename in datalist:
    print(filename)
    savefilename = getsavefilename(SavePath_afterag, filename)
    if not os.path.isfile(savefilename):
        Exptdata = np.load(filename)
        Exptdata = {key: Exptdata[key] for key in Exptdata}
        for para in ["ne", "S50", "sumpf"]:
            Exptdata[para] = np.log10(Exptdata[para])
        print("isgamma")
        Exptdata["isgamma"] = Pgamma_high_predictor.predict_proba(
            pd.DataFrame(Exptdata)
        )[1].to_numpy()
        isgammacut = np.where(Exptdata["isgamma"] > 0.1)
        Exptdata = {key: Exptdata[key][isgammacut] for key in Exptdata.keys()}
        print("energy")
        Exptdata["energy"] = (
            10 ** Energy_high_predictor.predict(pd.DataFrame(Exptdata)).to_numpy()
        )
        print("theta")
        Exptdata["deltatheta"] = deltatheta_high_predictor.predict(
            pd.DataFrame(Exptdata)
        ).to_numpy()
        Exptdata["theta_new"] = Exptdata["theta"] - Exptdata["deltatheta"]
        print("phi")
        Exptdata["deltaphi"] = deltaphi_high_predictor.predict(
            pd.DataFrame(Exptdata)
        ).to_numpy()
        Exptdata["phi_new"] = Exptdata["phi"] - Exptdata["deltaphi"]
        np.savez(savefilename, **Exptdata)

/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/003991.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004103.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004091.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004108.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004023.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004058.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004005.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004029.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004048.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/003972.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004060.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004094.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_merged/004047.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEne

In [3]:
def getRaDec(filename):
    savefilename = getsavefilename(SavePath_RaDec, filename)
    print(savefilename)
    if not os.path.isfile(savefilename):
        Exptdata = np.load(filename)
        Exptdata = {key: Exptdata[key] for key in Exptdata}
        Exptdata["theta_new"][Exptdata["theta_new"] < 0] = 0.01
        Exptdata["Ra"], Exptdata["Dec"], Exptdata["LST"], Exptdata["ha"] = corrdinateYBJ(
            Exptdata["theta_new"], Exptdata["phi_new"], Exptdata["mjd"]
        )
        np.savez(savefilename, **Exptdata)


def RaDecerrCallback(result):
    print(result)

In [7]:
mkdir(SavePath_RaDec)
pool = multiprocessing.Pool(processes=20)
for root, dirs, files in os.walk(SavePath_afterag):
    for name in files:
        filename = os.path.join(root, name)
        pool.apply_async(getRaDec, args=(filename,), error_callback=RaDecerrCallback)
pool.close()
pool.join()

/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003991.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004091.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004108.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004103.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004023.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004058.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004048.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003972.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004060.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004094.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004047.npz/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004021.npz


/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003964.npz

/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004022.np

/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004090.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003994.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004132.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004105.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003977.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003990.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004078.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003993.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004087.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004004.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004011.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004028.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/003994.npz
/home2/hky/github/Gamma_Energy/summary/Data/highEnergy_RaDec/004

In [10]:
Exptdata = dict()
for root, dirs, files in os.walk(SavePath_RaDec):
    for name in files:
        filename = os.path.join(root, name)
        Exptdata_tmp = np.load(filename)
        for key in Exptdata_tmp:
            if key not in Exptdata:
                Exptdata[key] = list()
            Exptdata[key].append(Exptdata_tmp[key])
for key in Exptdata.keys():
    Exptdata[key] = np.concatenate(Exptdata[key])
np.savez("./Exptdata_Ra_Dec.npz", **Exptdata)